In [ ]:
import random, os, time
import matplotlib.pyplot as plt
import numpy as np
from ctypes import *
from mpl_toolkits.mplot3d import Axes3D

%run "C:\Users\minet\notebook\python\encapsulate\encapsulateRBF.py"
%run "C:\Users\minet\notebook\python\encapsulate\encapsulateSharedMethods.py"
pathDLL = "C:/Users/minet/notebook/python/ML_Library.dll"

myDll = CDLL(pathDLL)

In [2]:
pathDatasetTrain = "C:/dataset/Bigdataset_Crop_Color_50x50/train/"
pathDatasetPredict = "C:/dataset/Bigdataset_Crop_Color_50x50/test/"

start_time = time.time()

imagesNameTrain = os.listdir(pathDatasetTrain)
imagesNamePredict = os.listdir(pathDatasetPredict)

# Parameters
numberImageTrain = len(imagesNameTrain)
component  = 3 # color component
imageW = 50
imageH = 50
dividedPixel = 1
inputCountPerSample = (imageW * imageH * component)
gamma = 1
k = 100
it = 100

selectedImagesTrain = random.sample(imagesNameTrain, numberImageTrain)
selectedImagesTrain = convertListToString(selectedImagesTrain, pathDatasetTrain)

pMatrixX, pMatrixY = prepareDataset(myDll, selectedImagesTrain, imageW, imageH, numberImageTrain, component, dividedPixel)


In [3]:
# Create & Allocate RBF Model
myDll.createRBFModel.argtypes = [ c_uint ]
myDll.createRBFModel.restype = c_void_p
pArrayWeight = myDll.createRBFModel(inputCountPerSample)

In [ ]:
# Fit RBF with regression version
myDll.fitRBFRegression.argtypes = [ c_void_p, c_void_p, c_void_p, c_double, c_int, c_int ]
error = myDll.fitRBFRegression( pArrayWeight, pMatrixX, pMatrixY, gamma, k, it)

In [ ]:
# Params function
myDll.predictRBFRegression.argtypes = [ c_void_p, c_void_p]
myDll.predictRBFRegression.restype = c_double

In [ ]:
numberImagePredict = len (imagesNamePredict)
selectedImagesPredict = random.sample(imagesNamePredict, numberImagePredict)
selectedImagesPredict = [pathDatasetPredict + el for el in selectedImagesPredict ]

average = 0

    for image in selectedImagesPredict:
        imageName = image[image.rfind("/")+1:]
        age = int(imageName[:imageName.find("_")])
        
        pMatrixXPredict, pMatrixYPredict = prepareDataset(myDll, image, imageW, imageH, 1, component, is255, -1, -1)
        pVectorXPredict = matrixToVector(myDll, pMatrixXPredict, imageW * imageH * component, 0)
        
        result = myDll.predictNaiveRBFClassification( pArrayWeight, pVectorXPredict )
        
        deleteDatasetMatrix(myDll,  pMatrixXPredict, pMatrixYPredict)
       
        print("age : ", age, "/ predicted : ", res)
        average += (round(res) - age)**2
        
    average =  average / len(tabSelectedImages)